# LangSmith Starter
LangSmith is a built-in observability service and platform that integrates very easily with LangChain. You don't need to use LangSmith for this course, but it can be very helpful in understanding what is happening, and we recommend using it beyond this course for general development with LangChain — with all of that in mind we would recommend spending a little bit of time to get familiar with LangSmith.

## Setting up LangSmith
LangSmith does require an API key, but it comes with a generous free tier. You can sign up an account and get your API key https://smith.langchain.com/.

When using LangSmith, we need to setup our environment variables and provide our API key, like so: LANGCHAIN_API_KEY

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "langchain-course"

In most cases, this is all we need to start seeing logs and traces in the LangSmith UI(https://smith.langchain.com/). By default, LangChain will trace LLM calls, chains, etc. We'll take a look at a quick example of this below.

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

Let's invoke our LLM and then see what happens in the LangSmith UI.

In [4]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BGOrZG6K3OljCRp5dtE2bi90xjjRF', 'finish_reason': 'stop', 'logprobs': None}, id='run-be0a0704-e070-48d6-9606-9df83d2d28a5-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

After this we should see that a new project (langchain-course) has been created in the LangSmith UI inside that project, we should see the trace from our LLM call:
![image.png](../assets/langsmith01.png)

## Tracing Non-langchain Code

In [ ]:
Langsmith can trace functions that are not part of Langchain, we just need to add the @traceable decorator. Let's try this for a few simple functions.

In [5]:
from langsmith import traceable
import random
import time

@traceable
def generate_random_number():
    return random.randint(0,100)

@traceable
def generate_string_delay(input_str: str):
    number = random.randint(1, 5)
    time.sleep(number)
    return f"{input_str} {number}"

@traceable
def random_error():
    number = random.randint(0,1)
    if number == 0:
        raise ValueError("Random error!")
    else:
        return "No error"

In [9]:
# Let's run these a few times and see what happens
from tqdm.auto import tqdm

for _ in tqdm(range(10)):
    generate_random_number()
    generate_string_delay("hello")
    try:
        random_error()
    except ValueError:
        pass

100%|██████████| 10/10 [00:28<00:00,  2.81s/it]


Those traces should now be visibile in the LangSmith UI, again under the same project:
![image.png](../assets/langsmith02.png)

In [10]:
from langsmith import traceable

@traceable(name="Chitchat Maker")
def error_generation_function(question: str):
    delay = random.randint(0, 3)
    time.sleep(delay)
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error!")
    else:
        return f"Chitchat: {question} {delay}"

In [11]:
for _ in tqdm(range(10)):
    try:
        error_generation_function("How are you today?")
    except ValueError:
        pass

100%|██████████| 10/10 [00:13<00:00,  1.31s/it]


![image.png](../assets/langsmith03.png)